# City street network orientations

Compare the spatial orientations of city street networks with OSMnx.

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

ox.config(log_console=True, use_cache=True)
weight_by_length = False

ox.__version__

'0.10'

In [2]:
# define the study sites as label : query
# the full list takes ~30 minutes
# # places = {'Atlanta'       : 'Atlanta, GA, USA',
#           'Boston'        : 'Boston, MA, USA',
#           'Buffalo'       : 'Buffalo, NY, USA',
#           'Charlotte'     : 'Charlotte, NC, USA',
#           'Chicago'       : 'Chicago, IL, USA',
#           'Cleveland'     : 'Cleveland, OH, USA',
#           'Dallas'        : 'Dallas, TX, USA',
#           'Houston'       : 'Houston, TX, USA',
#           'Denver'        : 'Denver, CO, USA',
#           'Detroit'       : 'Detroit, MI, USA',
#           'Las Vegas'     : 'Las Vegas, NV, USA',
#           'Los Angeles'   : {'city':'Los Angeles', 'state':'CA', 'country':'USA'},
#           'Manhattan'     : 'Manhattan, NYC, NY, USA',
#           'Miami'         : 'Miami, FL, USA',
#           'Minneapolis'   : 'Minneapolis, MN, USA',
#           'Orlando'       : 'Orlando, FL, USA',
#           'Philadelphia'  : 'Philadelphia, PA, USA',
#           'Phoenix'       : 'Phoenix, AZ, USA',
#           'Portland'      : 'Portland, OR, USA',
#           'Sacramento'    : 'Sacramento, CA, USA',
#           'San Francisco' : {'city':'San Francisco', 'state':'CA', 'country':'USA'},
#           'Seattle'       : 'Seattle, WA, USA',
#           'St Louis'      : 'St. Louis, MO, USA',
#           'Tampa'         : 'Tampa, FL, USA',
#           'Washington'    : 'Washington, DC, USA'}

# reduced list. Runtime ~5 mins. 
places = {'Boston'        : 'Boston, MA, USA',
          'Charlotte'     : 'Charlotte, NC, USA',
          'Detroit'       : 'Detroit, MI, USA',
          'Manhattan'     : 'Manhattan, NYC, NY, USA',
          'Sacramento'    : 'Sacramento, CA, USA',
          'St Louis'      : 'St. Louis, MO, USA'}

In [3]:
# verify OSMnx geocodes each query to what you expect
gdf = ox.gdf_from_places(places.values())
gdf

,bbox_east,bbox_north,bbox_south,bbox_west,geometry,place_name
0,-70.804488,42.396977,42.227911,-71.191260,"POLYGON ((-71.19126 42.2828375, -71.1910708000...","Boston, Suffolk County, Massachusetts, USA"
1,-80.670104,35.393133,35.013174,-81.009554,"POLYGON ((-81.00955399999999 35.151438, -81.00...","Charlotte, Mecklenburg County, North Carolina,..."
2,-82.910439,42.450243,42.255192,-83.287959,"POLYGON ((-83.287959 42.44268, -83.287852 42.4...","Detroit, Wayne County, Michigan, USA"
3,-73.910187,40.877796,40.699682,-74.019442,"POLYGON ((-74.01944159999999 40.7060712, -74.0...","Manhattan, New York County, New York City, New..."
4,-121.362740,38.685506,38.437574,-121.560120,"POLYGON ((-121.56012 38.656339, -121.559298 38...","Sacramento, Sacramento County, California, USA"
5,-90.166292,38.774340,38.532322,-90.320652,"POLYGON ((-90.32065249999999 38.594204, -90.31...","St. Louis, City of St. Louis, Missouri, USA"


## Get the street networks and their edge bearings

In [4]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

In [5]:
from datetime import datetime
start=datetime.now()

bearings = {}
for place in sorted(places.keys()):
    
    # get the graph
    query = places[place]
    G = ox.graph_from_place(query, network_type='drive')
    
    # calculate edge bearings
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    
    if weight_by_length:
        # weight bearings by length (meters)
        city_bearings = []
        for u, v, k, d in Gu.edges(keys=True, data=True):
            city_bearings.extend([d['bearing']] * int(d['length']))
        b = pd.Series(city_bearings)
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
        
    else:
        # don't weight bearings, just take one value per street segment
        b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    
    print(place, ": ", datetime.now()-start, "elapsed")
        
print("total time: ", datetime.now()-start)

Boston :  0:00:29.102260 elapsed
Charlotte :  0:01:58.232247 elapsed
Detroit :  0:02:45.135833 elapsed
Manhattan :  0:02:54.344558 elapsed
Sacramento :  0:03:29.973197 elapsed
St Louis :  0:03:48.838284 elapsed
total time:  0:03:48.839283


## Visualize it

In [6]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

In [7]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):

    bins = np.arange(n + 1) * 360 / n
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'Century Gothic', 'size':24, 'weight':'bold'}
    xtick_font = {'family':'Century Gothic', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'Century Gothic', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [8]:
# create figure and axes

# leaves blank figures where list is not a multiple of 3. 

n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})

# plot each city's polar histogram
for ax, place in zip(axes.flat, sorted(places.keys())):
    polar_plot(ax, bearings[place].dropna(), title=place)

# add super title and save full image
suptitle_font = {'family':'Century Gothic', 'fontsize':60, 'fontweight':'normal', 'y':1.07}
fig.suptitle('City Street Network Orientation', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
fig.savefig('images/street-orientations.png', dpi=120, bbox_inches='tight')
plt.close()